# Temporal Workflows

This notebook demonstrates durable workflow execution with Temporal.

## Prerequisites

```bash
pip install neon-sdk[temporal]
```

Start Temporal with Docker:

```bash
docker compose --profile temporal up -d
```

In [ ]:
# Install if needed
# !pip install neon-sdk[temporal]

## 1. Client Setup

In [ ]:
from neon_sdk.temporal import NeonTemporalClient, TemporalClientConfig

# Create client
client = NeonTemporalClient(TemporalClientConfig(
    address="localhost:7233",
    namespace="default",
    task_queue="agent-workers",
))

print("Temporal client created")
print(f"Address: {client.config.address}")
print(f"Namespace: {client.config.namespace}")
print(f"Task Queue: {client.config.task_queue}")

## 2. Connect to Temporal

In [ ]:
# Connect (uncomment when Temporal is running)
# await client.connect()
# print("Connected to Temporal!")

print("Would connect to Temporal at localhost:7233")

## 3. Starting an Agent Run

In [ ]:
from neon_sdk.temporal import StartAgentRunInput

# Define the agent run input
agent_input = StartAgentRunInput(
    project_id="demo-project",
    agent_id="customer-support-agent",
    agent_version="1.0.0",
    input_data={
        "query": "How do I reset my password?",
        "user_id": "user-123",
        "context": {
            "previous_interactions": 3,
            "sentiment": "neutral",
        },
    },
    tools=[
        {"name": "knowledge_base", "type": "retrieval"},
        {"name": "send_email", "type": "action"},
    ],
    timeout_seconds=300,
    metadata={"source": "notebook"},
)

print("Agent run input configured:")
print(f"  Project: {agent_input.project_id}")
print(f"  Agent: {agent_input.agent_id}")
print(f"  Query: {agent_input.input_data['query']}")

# Start the agent run (uncomment when Temporal is running)
# result = await client.start_agent_run(agent_input)
# print(f"\nWorkflow started!")
# print(f"  Workflow ID: {result['workflow_id']}")
# print(f"  Run ID: {result['run_id']}")

## 4. Checking Agent Status

In [ ]:
# Check status (uncomment when Temporal is running)
# status = await client.get_agent_status(workflow_id)
# 
# print(f"Agent Status")
# print(f"============")
# print(f"Status: {status.status}")
# print(f"Started: {status.started_at}")
# print(f"Completed: {status.completed_at}")

print("Agent Status (example)")
print("======================")
print("Status: running")
print("Started: 2024-01-15T10:30:00Z")
print("Completed: None (still running)")

## 5. Getting Progress Updates

In [ ]:
# Get progress (uncomment when Temporal is running)
# progress = await client.get_agent_progress(workflow_id)
# 
# print(f"Progress: Step {progress.current_step}/{progress.total_steps}")
# print(f"Current action: {progress.current_action}")
# print(f"\nMessages:")
# for msg in progress.messages:
#     print(f"  [{msg.timestamp}] {msg.content}")

print("Agent Progress (example)")
print("========================")
print("Progress: Step 2/4")
print("Current action: Searching knowledge base")
print("")
print("Messages:")
print("  [10:30:01] Received query about password reset")
print("  [10:30:02] Searching knowledge base for relevant articles...")
print("  [10:30:05] Found 3 relevant articles")

## 6. Human-in-the-Loop Approval

In [ ]:
# Approve or reject agent action (uncomment when Temporal is running)
# 
# Approve:
# await client.approve_agent(workflow_id, approved=True)
# print("Agent approved to continue")
# 
# Reject:
# await client.approve_agent(
#     workflow_id,
#     approved=False,
#     reason="Response needs revision",
# )
# print("Agent rejected - will retry")

print("Human-in-the-Loop Example")
print("=========================")
print("Agent is waiting for approval...")
print("")
print("Proposed action:")
print("  Send password reset email to user@example.com")
print("")
print("Options:")
print("  - approve_agent(workflow_id, approved=True)")
print("  - approve_agent(workflow_id, approved=False, reason='...')")

## 7. Waiting for Results

In [ ]:
# Wait for final result (uncomment when Temporal is running)
# try:
#     result = await client.wait_for_agent_result(
#         workflow_id,
#         timeout_seconds=300,
#     )
#     print(f"Agent completed!")
#     print(f"Output: {result.output}")
#     print(f"Duration: {result.duration_ms}ms")
# except TimeoutError:
#     print("Agent timed out")

print("Agent Result (example)")
print("======================")
print("Status: completed")
print("Duration: 4,523ms")
print("")
print("Output:")
print("  response: 'I can help you reset your password. I've sent a reset link to your email.'")
print("  actions_taken: ['search_kb', 'generate_response', 'send_email']")
print("  confidence: 0.95")

## 8. Running Evaluations

In [ ]:
from neon_sdk.temporal import StartEvalRunInput

# Define evaluation input
eval_input = StartEvalRunInput(
    run_id="eval-001",
    project_id="demo-project",
    agent_id="customer-support-agent",
    agent_version="1.0.0",
    dataset={
        "items": [
            {
                "input": {"query": "How do I reset my password?"},
                "expected": {"contains": ["password", "reset", "email"]},
            },
            {
                "input": {"query": "What are your business hours?"},
                "expected": {"contains": ["hours", "open"]},
            },
            {
                "input": {"query": "How do I cancel my subscription?"},
                "expected": {"contains": ["cancel", "subscription"]},
            },
        ]
    },
    tools=[
        {"name": "knowledge_base", "type": "retrieval"},
    ],
    scorers=["contains", "response_quality", "latency"],
    concurrency=3,  # Run 3 cases in parallel
)

print("Evaluation configured:")
print(f"  Test cases: {len(eval_input.dataset['items'])}")
print(f"  Scorers: {', '.join(eval_input.scorers)}")
print(f"  Concurrency: {eval_input.concurrency}")

# Start evaluation (uncomment when Temporal is running)
# result = await client.start_eval_run(eval_input)
# print(f"\nEval started: {result['workflow_id']}")

## 9. Monitoring Eval Progress

In [ ]:
# Monitor progress (uncomment when Temporal is running)
# progress = await client.get_eval_progress(eval_workflow_id)
# 
# print(f"Eval Progress")
# print(f"=============")
# print(f"Completed: {progress.completed}/{progress.total}")
# print(f"Passed: {progress.passed}")
# print(f"Failed: {progress.failed}")
# print(f"Avg score: {progress.avg_score:.2f}")

print("Eval Progress (example)")
print("=======================")
print("Completed: 2/3")
print("Passed: 2")
print("Failed: 0")
print("Avg score: 0.87")
print("")
print("Cases:")
print("  [PASS] Case 1: 0.92")
print("  [PASS] Case 2: 0.82")
print("  [    ] Case 3: running...")

## 10. Getting Eval Results

In [ ]:
# Get final results (uncomment when Temporal is running)
# results = await client.get_eval_results(eval_workflow_id)
# 
# print(f"Evaluation Results")
# print(f"==================")
# print(f"Overall score: {results.overall_score:.2f}")
# print(f"Duration: {results.duration_ms}ms")
# print(f"")
# print(f"Scorer Statistics:")
# for name, stats in results.scorer_stats.items():
#     print(f"  {name}:")
#     print(f"    Avg: {stats.avg:.2f}")
#     print(f"    Min: {stats.min:.2f}")
#     print(f"    Max: {stats.max:.2f}")

print("Evaluation Results (example)")
print("============================")
print("Overall score: 0.85")
print("Duration: 12,450ms")
print("")
print("Scorer Statistics:")
print("  contains:")
print("    Avg: 1.00")
print("    Min: 1.00")
print("    Max: 1.00")
print("")
print("  response_quality:")
print("    Avg: 0.82")
print("    Min: 0.75")
print("    Max: 0.89")
print("")
print("  latency:")
print("    Avg: 0.72")
print("    Min: 0.65")
print("    Max: 0.80")

## 11. Listing Workflows

In [ ]:
# List workflows (uncomment when Temporal is running)
# workflows = await client.list_workflows(
#     query="WorkflowType='agentRunWorkflow'",
# )
# 
# print(f"Found {len(workflows)} workflows")
# for wf in workflows[:5]:
#     print(f"  {wf.workflow_id}: {wf.status}")

print("Recent Workflows (example)")
print("==========================")
print("  agent-run-001: completed")
print("  agent-run-002: running")
print("  eval-001: completed")
print("  agent-run-003: failed")
print("  agent-run-004: pending")

## 12. Error Handling

In [ ]:
from neon_sdk.temporal import TemporalError, WorkflowNotFoundError

# Example error handling
# try:
#     status = await client.get_agent_status("non-existent-workflow")
# except WorkflowNotFoundError:
#     print("Workflow not found")
# except TemporalError as e:
#     print(f"Temporal error: {e}")

print("Error Handling Example")
print("======================")
print("")
print("try:")
print("    status = await client.get_agent_status(workflow_id)")
print("except WorkflowNotFoundError:")
print("    print('Workflow not found')")
print("except TemporalError as e:")
print("    print(f'Temporal error: {e}')")

## 13. Cleanup

In [ ]:
# Disconnect when done (uncomment when Temporal is running)
# await client.disconnect()
# print("Disconnected from Temporal")

print("Cleanup: await client.disconnect()")

## Next Steps

- [Documentation](https://neon-sdk.readthedocs.io/en/latest/guides/temporal.html)
- [Temporal Web UI](http://localhost:8080) - Monitor workflows visually
- [Temporal Documentation](https://docs.temporal.io)